In [ ]:
# 📚 Celda 1: EnvironmentDetector con ejecución inmediata

import platform
import os

class EnvironmentDetector:
    """Detecta información básica del entorno operativo."""

    def __init__(self):
        self.system = platform.system()
        self.release = platform.release()
        self.version = platform.version()
        self.architecture = platform.machine()
        self.in_docker = self._detect_docker()

    def _detect_docker(self) -> bool:
        """Detecta si el programa corre dentro de un contenedor Docker."""
        try:
            if os.path.exists('/.dockerenv'):
                return True
            if os.path.isfile('/proc/1/cgroup'):
                with open('/proc/1/cgroup', 'rt') as f:
                    return any('docker' in line for line in f)
        except Exception:
            pass
        return False

    def get_environment_info(self) -> dict:
        """Retorna la información del entorno."""
        return {
            "system": self.system,
            "release": self.release,
            "version": self.version,
            "architecture": self.architecture,
            "in_docker": self.in_docker
        }

# 🔥 Ejecución inmediata al final de la celda

detector = EnvironmentDetector()
env_info = detector.get_environment_info()

print("🖥️ Entorno detectado automáticamente:")
for key, value in env_info.items():
    print(f"  {key}: {value}")


In [ ]:
# 📚 Celda 2: Configuración Inicial para el Orquestador

CONFIG = {
    "retry_attempts": 3,         # Intentos de reconexión en caso de fallo
    "retry_delay_sec": 5,         # Segundos de espera entre reintentos
    "monitor_interval_sec": 30,   # Intervalo de chequeo de conexiones (en segundos)
    "default_mysql_port": 3306,   # Puerto por defecto para MySQL
    "default_postgresql_port": 5432,  # Puerto por defecto para PostgreSQL
    "default_sqlite_memory": ":memory:",  # Base de datos en memoria para SQLite
}

# 🔥 Imprimir configuración automáticamente

print("⚙️ Configuración inicial cargada:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


In [ ]:
# 📚 Celda 3: Definición inicial del DBOrchestrator

class DBOrchestrator:
    """Orquestador de bases de datos y entorno del sistema."""

    def __init__(self, config: dict):
        print("🚀 Inicializando DB Orchestrator...\n")
        
        # Guardar configuración
        self.config = config
        
        # Detectar entorno
        self.env_detector = EnvironmentDetector()
        self.environment_info = self.env_detector.get_environment_info()
        
        # Estado interno
        self.connections = {}           # Guardará conexiones a bases de datos
        self.detected_db_systems = []    # Guardará sistemas de DB encontrados (cuando escaneemos)
        self.active_connection_id = None # ID de la conexión activa principal

        # Mostrar entorno detectado y configuración cargada
        self._print_environment_info()
        self._print_config()

    def _print_environment_info(self):
        """Imprime la información detectada del entorno."""
        print("🖥️ Entorno operativo detectado:")
        for key, value in self.environment_info.items():
            print(f"  {key}: {value}")
        print("")  # Espacio para separar bloques

    def _print_config(self):
        """Imprime la configuración activa."""
        print("⚙️ Configuración inicial activa:")
        for key, value in self.config.items():
            print(f"  {key}: {value}")
        print("")  # Espacio para separar bloques

print("📦 Clase DBOrchestrator definida correctamente y lista para usar.")


In [ ]:
# 📚 Celda 4: Instanciación del DBOrchestrator y visualización completa

# Crear el orquestador usando la configuración
orchestrator = DBOrchestrator(config=CONFIG)

# Preparar datos para mostrar
environment_info = orchestrator.environment_info
config_info = orchestrator.config

# Mostrar visualmente los valores
print("\n🖥️ Entorno operativo final detectado:")
for key, value in environment_info.items():
    print(f"  {key}: {value}")

print("\n⚙️ Configuración final activa:")
for key, value in config_info.items():
    print(f"  {key}: {value}")

# 🔥 Devolver explícitamente para lectura posterior si se quiere analizar en notebooks
(environment_info, config_info)


In [ ]:
# 📚 Celda 5: Detección de cliente MySQL y escaneo de puertos

import socket
import psutil

def is_mysql_client_available():
    """Detecta si el cliente mysql.connector está instalado."""
    try:
        import mysql.connector
        return True
    except ImportError:
        return False

def scan_ports_for_mysql(possible_ports=[3306, 3307, 3308, 3309]):
    """Escanea puertos locales típicos buscando un servidor MySQL."""
    open_ports = []
    
    for port in possible_ports:
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=1):
                open_ports.append(port)
        except (socket.timeout, ConnectionRefusedError, OSError):
            continue  # No hay servidor escuchando en este puerto
    
    return open_ports

def detect_mysql_activity():
    """Detecta cliente de MySQL y servidores activos en puertos locales."""
    mysql_client = is_mysql_client_available()
    mysql_ports = []

    # Intentar usar psutil si disponible
    try:
        for conn in psutil.net_connections(kind='inet'):
            if conn.status == psutil.CONN_LISTEN:
                port = conn.laddr.port
                pid = conn.pid
                
                if pid is None:
                    continue
                
                try:
                    process = psutil.Process(pid)
                    name = process.name().lower()
                    if 'mysql' in name or 'mariadb' in name:
                        mysql_ports.append(port)
                except (psutil.NoSuchProcess, psutil.AccessDenied):
                    continue
    except Exception:
        pass

    # Si no encontramos puertos o queremos doble confirmación, escanear directamente
    scanned_ports = scan_ports_for_mysql()
    for port in scanned_ports:
        if port not in mysql_ports:
            mysql_ports.append(port)
    
    return mysql_client, sorted(set(mysql_ports))

# 🔥 Ejecutar la detección
mysql_client_found, mysql_ports_found = detect_mysql_activity()

# 🔥 Mostrar resultados
print("🔎 Resultado de la detección de MySQL:")

if mysql_client_found:
    print("✅ Cliente de MySQL (mysql.connector) disponible.")
else:
    print("❌ Cliente de MySQL (mysql.connector) NO disponible.")

if mysql_ports_found:
    print(f"🚪 Puertos de MySQL detectados: {mysql_ports_found}")
else:
    print("⚠️ No se detectaron puertos de MySQL activos.")
    
# Devolver para futuras celdas
(mysql_client_found, mysql_ports_found)


In [ ]:
# 📚 Celda 6: Detección de cliente PostgreSQL y escaneo de puertos

import socket

def is_postgresql_client_available():
    """Detecta si el cliente psycopg2 está instalado."""
    try:
        import psycopg2
        return True
    except ImportError:
        return False

def scan_ports_for_postgresql(possible_ports=[5432, 5433, 5434]):
    """Escanea puertos locales típicos buscando un servidor PostgreSQL."""
    open_ports = []
    
    for port in possible_ports:
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=1):
                open_ports.append(port)
        except (socket.timeout, ConnectionRefusedError, OSError):
            continue  # No hay servidor escuchando en este puerto
    
    return open_ports

def detect_postgresql_activity():
    """Detecta cliente de PostgreSQL y servidores activos en puertos locales."""
    postgresql_client = is_postgresql_client_available()
    postgresql_ports = scan_ports_for_postgresql()
    
    return postgresql_client, sorted(set(postgresql_ports))

# 🔥 Ejecutar la detección
postgresql_client_found, postgresql_ports_found = detect_postgresql_activity()

# 🔥 Mostrar resultados
print("🔎 Resultado de la detección de PostgreSQL:")

if postgresql_client_found:
    print("✅ Cliente de PostgreSQL (psycopg2) disponible.")
else:
    print("❌ Cliente de PostgreSQL (psycopg2) NO disponible.")

if postgresql_ports_found:
    print(f"🚪 Puertos de PostgreSQL detectados: {postgresql_ports_found}")
else:
    print("⚠️ No se detectaron puertos de PostgreSQL activos.")
    
# Devolver para futuras celdas
(postgresql_client_found, postgresql_ports_found)


In [ ]:
# 📚 Celda 7: Ficha completa de proveedores de base de datos detectados

# Detectar SQLite (siempre disponible en Python)
sqlite_client_found = True
sqlite_ports_found = []  # SQLite no usa puertos TCP

# 🔥 Crear reporte
database_summary = {
    "MySQL": {
        "client_available": mysql_client_found,
        "ports_detected": mysql_ports_found
    },
    "PostgreSQL": {
        "client_available": postgresql_client_found,
        "ports_detected": postgresql_ports_found
    },
    "SQLite": {
        "client_available": sqlite_client_found,
        "ports_detected": sqlite_ports_found  # No aplica
    }
}

# 🔥 Mostrar reporte de forma clara
print("🗂️ Ficha de proveedores de base de datos disponibles:\n")

for db_name, info in database_summary.items():
    print(f"🔵 {db_name}")
    print(f"   - Cliente disponible: {'✅' if info['client_available'] else '❌'}")
    
    if info['ports_detected']:
        print(f"   - Puertos detectados: {info['ports_detected']}")
    else:
        if db_name == "SQLite":
            print("   - (No necesita puertos, funciona internamente)")
        else:
            print("   - No se detectaron puertos activos.")
    print("")  # Espacio entre bases

# 🔥 Devolver resumen por si queremos usarlo después
database_summary


In [ ]:
# 📚 Nueva celda: Definición de generate_database_summary()

def generate_database_summary():
    """Genera un resumen completo de todos los proveedores detectados."""
    
    # Detectar MySQL
    mysql_client, mysql_ports = detect_mysql_activity()
    
    # Detectar PostgreSQL
    postgresql_client, postgresql_ports = detect_postgresql_activity()
    
    # Detectar SQLite (siempre disponible)
    sqlite_client = True
    sqlite_ports = []  # No usa puertos
    
    # Armar el resumen
    summary = {
        "MySQL": {
            "client_available": mysql_client,
            "ports_detected": mysql_ports
        },
        "PostgreSQL": {
            "client_available": postgresql_client,
            "ports_detected": postgresql_ports
        },
        "SQLite": {
            "client_available": sqlite_client,
            "ports_detected": sqlite_ports
        }
    }
    
    return summary


In [ ]:
# 📚 Celda 8: Generar ficha completa del sistema

# Reusar EnvironmentDetector
detector = EnvironmentDetector()
environment_info = detector.get_environment_info()

# Reusar CONFIG
configurations = CONFIG

# Reusar la generación de proveedores de bases de datos
database_summary = generate_database_summary()

# 🔥 Ensamblar todo en una sola ficha
full_system_summary = {
    "system_info": environment_info,
    "configurations": configurations,
    "database_providers": database_summary
}

# 🔥 Mostrar la ficha completa de forma bonita
print("🗂️ Ficha completa del sistema detectado:\n")

# Mostrar información del sistema
print("🔵 Entorno operativo:")
for key, value in environment_info.items():
    print(f"   {key}: {value}")

print("\n🔵 Configuraciones activas:")
for key, value in configurations.items():
    print(f"   {key}: {value}")

print("\n🔵 Proveedores de base de datos detectados:")
for db_name, info in database_summary.items():
    print(f"\n   🔹 {db_name}")
    print(f"     - Cliente disponible: {'✅' if info['client_available'] else '❌'}")
    if info['ports_detected']:
        print(f"     - Puertos detectados: {info['ports_detected']}")
    else:
        if db_name == "SQLite":
            print(f"     - (No necesita puertos, funciona internamente)")
        else:
            print(f"     - No se detectaron puertos activos.")

# 🔥 Devolver la ficha completa como variable por si quieres usarla
full_system_summary
